In [74]:
# ライブラリの導入
import pandas as pd
import biogeme.database as db
from biogeme.expressions import Beta, Variable
from biogeme.expressions import exp
from biogeme.mdcev import GammaProfile
from biogeme.mdcev.database_utils import mdcev_count
# 20260105時点で出てくる警告はGPUが使えないから少し遅いかもというもの．GPUを使えるようにするにはいくつか処置が必要

In [75]:
# データの読み込み
di_master_df = pd.read_csv("/home/shibumtk/B4research/estimate/data/01master_data/MDCEV_di_master1.csv", encoding="utf-8-sig")
# 文字列は除外する
drop_cols =['NearestStation', 'WorkplaceStation']
di_master_df = di_master_df.drop(columns=drop_cols)

print(len(di_master_df))
di_master_df.columns

3462


Index(['Personal_ID', 'HouseholdIncome', 'sex', 'age', 'JobType',
       'ComuTime[m]', 'MainlineTime[m]', 'AccessTime_used[m]', 'HouseholdType',
       'HouseholdMembers(all)', 'YoungestMember_No', 'WorkTime[m]',
       'AfterTime_work[m]', 'AfterTime_home[m]', 'first_transportation',
       'NearestStation_code', 'near_area_syou', 'near_area_kinrin',
       'near_area_sum', 'near_area_syou500', 'near_area_kinrin500',
       'near_area_sum500', 'near_area_syou300', 'near_area_kinrin300',
       'near_area_sum300', 'near_o_area_syou', 'near_o_area_kinrin',
       'near_o_area_sum', 'near_o_area_syou500', 'near_o_area_kinrin500',
       'near_o_area_sum500', 'near_o_area_syou300', 'near_o_area_kinrin300',
       'near_o_area_sum300', 'WorkplaceStation_code', 'WP_area_syou',
       'WP_area_kinrin', 'WP_area_sum', 'WP_area_syou500', 'WP_area_kinrin500',
       'WP_area_sum500', 'WP_area_syou300', 'WP_area_kinrin300',
       'WP_area_sum300', 'WP_o_area_syou', 'WP_o_area_kinrin', 'WP_o_ar

In [76]:
df = di_master_df
(df['AfterTime_home[m]'] <= 0).sum(), df['AfterTime_home[m]'].min()

(np.int64(0), np.float64(190.0))

In [77]:
# Biogemeデータベースに格納
database = db.Database("di_mdcev", di_master_df)

In [ ]:
# 変数の定義
# 滞在時間（分）
AFTER_HOME_MIN = Variable('AfterTime_home[m]')   # 自宅
T1_MIN = Variable('timing1_time')
T2_MIN = Variable('timing2_time')
T3_MIN = Variable('timing3_time')
T4_MIN = Variable('timing4_time')

# 個人属性・アクセシビリティ
WORKTIME    = Variable('WorkTime[m]')
AF_WORKTIME = Variable("AfterTime_work[m]") # AF=after
MACMTIME    = Variable('MainlineTime[m]')
HHM_all     = Variable('HouseholdMembers(all)')
NEKI_INDEX  = Variable('near_area_syou300') # 自宅最寄駅土地利用指標
WEKI_INDEX  = Variable("WP_area_syou300") # 就業地駅土地利用指標

# 実際に利用した端末モード
AC_MODE = Variable('first_transportation')

# traffic_mode
WA_AV = Variable("walk_av")
WA_TT = Variable("walk_time[m]")
BI_AV = Variable("bicycle_av")
BI_TT = Variable("bicycle_time[m]")
BU_AV = Variable("bus_av")
BU_TT = Variable("bus_time[m]")
CA_AV = Variable("car_av")
CA_TT = Variable("car_time[m]")

# scaled
WORKTIME_SCALED = database.define_variable('WORKTIME_SCALED', WORKTIME / 60.0)
MACMTIME_SCALED = database.define_variable('MACMTIME_SCALED', MACMTIME / 60.0)

NEKI_INDEX_SCALED = database.define_variable('NEKI_INDEX_SCALED', NEKI_INDEX / 100.0)
WEKI_INDEX_SCALED = database.define_variable('WEKI_INDEX_SCALED', WEKI_INDEX / 100.0)

# 実際に利用したモードのダミー（徒歩基準）
USED_WALK = database.define_variable('USED_WALK', (AC_MODE == 1))
USED_BIKE = database.define_variable('USED_BIKE', (AC_MODE == 2))
USED_BUS  = database.define_variable('USED_BUS',  (AC_MODE == 3))
USED_CAR  = database.define_variable('USED_CAR',  (AC_MODE == 4))

# 代表時間の定義
REP_TTIME_MIN = database.define_variable(
    'REP_TTIME_MIN',
    (USED_WALK * WA_TT +
     USED_BIKE * BI_TT +
     USED_BUS  * BU_TT +
     USED_CAR  * CA_TT))

# 選択された財の個数を作成（正の滞在時間の個数）
mdcev_count(
    database,
    ['AfterTime_home[m]', 'timing1_time', 'timing2_time',
     'timing3_time', 'timing4_time'],
    new_column='number_chosen',
)
NUMBER_CHOSEN = Variable('number_chosen')

# parameters
# alternative specific constant
ASC_HOME = Beta('ASC_HOME', 0, None, None, 1)  # 自宅を基準として 0 に固定
ASC_T1   = Beta('ASC_T1',   0, None, None, 0)
ASC_T2   = Beta('ASC_T2',   0, None, None, 0)
ASC_T3   = Beta('ASC_T3',   0, None, None, 0)
ASC_T4   = Beta('ASC_T4',   0, None, None, 0)

# common coef
B_WORK = Beta('B_WORK', 0, None, None, 0)
D_WORK_T1 = Beta('D_WORK_T1', 0, None, None, 0)
D_WORK_T3 = Beta('D_WORK_T3', 0, None, None, 0)
D_WORK_T4 = Beta('D_WORK_T4', 0, None, None, 0)
# t2 は基準なので偏差なし（=0）

B_AFTI = Beta('B_AFTI', 0, None, None, 0)

B_MACM = Beta('B_MACM', 0, None, None, 0)
D_MACM_T1 = Beta('D_MACM_T1', 0, None, None, 0)
D_MACM_T3 = Beta('D_MACM_T3', 0, None, None, 0)
D_MACM_T4 = Beta('D_MACM_T4', 0, None, None, 0)

B_HHM  = Beta('B_HHM',  0, None, None, 0)
B_NEKI = Beta('B_NEKI', 0, None, None, 0)
B_WEKI = Beta('B_WEKI', 0, None, None, 0)

# mode coef（徒歩基準）
B_AT     = Beta('B_AT',    0, None, None, 0)  # 共通時間係数
D_AT_T1  = Beta('D_AT_T1', 0, None, None, 0)
D_AT_T3  = Beta('D_AT_T3', 0, None, None, 0)
D_AT_T4  = Beta('D_AT_T4', 0, None, None, 0)

# Utility
# station part
NEKI_PART = B_NEKI * NEKI_INDEX_SCALED
WEKI_PART = B_WEKI * WEKI_INDEX_SCALED

# 各選択肢に対する効用関数
# 自宅（基準財）は「外出しない」ので、まずは TIME_TERM を入れない仕様にしておきます。
home = ASC_HOME

# 外出先4タイミングでは、共通部分 + 代表時間効果を入れる
t1 = (ASC_T1 
   + (B_WORK + D_WORK_T1) * WORKTIME_SCALED 
   + (B_MACM + D_MACM_T1) * MACMTIME_SCALED 
   + B_HHM * HHM_all 
   + (B_AT + D_AT_T1) * REP_TTIME_MIN
   + WEKI_PART)

t2 = (ASC_T2 
   + (B_WORK) * WORKTIME_SCALED 
   + (B_MACM) * MACMTIME_SCALED 
   + B_HHM * HHM_all 
   + (B_AT) * REP_TTIME_MIN)

t3 = (ASC_T3 
   + (B_WORK + D_WORK_T3) * WORKTIME_SCALED 
   + (B_MACM + D_MACM_T3) * MACMTIME_SCALED 
   + B_HHM * HHM_all 
   + (B_AT + D_AT_T3) * REP_TTIME_MIN
   + NEKI_PART)

t4 = (ASC_T4 
   + (B_WORK + D_WORK_T4) * WORKTIME_SCALED 
   + (B_MACM + D_MACM_T4) * MACMTIME_SCALED 
   + B_HHM * HHM_all 
   + (B_AT + D_AT_T4) * REP_TTIME_MIN)


baseline_utilities = {
    1: home,  # AfterTime_home
    2: t1,    # timing1
    3: t2,    # timing2
    4: t3,    # timing3
    5: t4,    # timing4
}

# γパラメータとスケールパラメータ
lowest_positive_value = 0.0001

gamma_home = Beta('gamma_home', 1, lowest_positive_value, None, 0)
gamma_t1   = Beta('gamma_t1',   1, lowest_positive_value, None, 0)
gamma_t2   = Beta('gamma_t2',   1, lowest_positive_value, None, 0)
gamma_t3   = Beta('gamma_t3',   1, lowest_positive_value, None, 0)
gamma_t4   = Beta('gamma_t4',   1, lowest_positive_value, None, 0)

LOG_GAMMA_HOME = Beta('LOG_GAMMA_HOME', 0, None, None, 0)
LOG_GAMMA_OUT  = Beta('LOG_GAMMA_OUT',  0, None, None, 0)

gamma_home = exp(LOG_GAMMA_HOME)
gamma_out  = exp(LOG_GAMMA_OUT)

gamma_parameters = {
    1: gamma_home,
    2: gamma_out,
    3: gamma_out,
    4: gamma_out,
    5: gamma_out,
}

scale_parameter = Beta('scale', 1, lowest_positive_value, None, 0)

# 消費量
consumed_quantities = {
    1: AFTER_HOME_MIN,
    2: T1_MIN,
    3: T2_MIN,
    4: T3_MIN,
    5: T4_MIN,
}

In [79]:
# モデルの推定と結果の表示
the_gamma_profile = GammaProfile(
    model_name='gamma_profile_afterwork',
    baseline_utilities=baseline_utilities,
    gamma_parameters=gamma_parameters,
    scale_parameter=scale_parameter,
    # weights=weight,  # 標本ウェイトがあれば Variable('weight') をここに
)

results = the_gamma_profile.estimate_parameters(
    database=database,
    number_of_chosen_alternatives=NUMBER_CHOSEN,
    consumed_quantities=consumed_quantities,
    generate_html=False,
    generate_yaml=False,
)

print(results.short_summary())

pandas_results = results.get_estimated_parameters()
print(pandas_results)

It seems that the optimization algorithm did not converge. Therefore, the results may not correspond to the maximum likelihood estimator. Check the specification of the model, or the criteria for convergence of the algorithm.


Results for model gamma_profile_afterwork
Nbr of parameters:		22
Sample size:			3462
Excluded data:			0
Final log likelihood:		-25157.66
Akaike Information Criterion:	50359.32
Bayesian Information Criterion:	50494.61

              Name     Value  Robust std err.  Robust t-stat.  Robust p-value
0            scale  6.055821     8.544573e-02       70.873303    0.000000e+00
1   LOG_GAMMA_HOME  0.540739     1.639983e-01        3.297224    9.764545e-04
2           ASC_T1 -5.872086     1.709608e-01      -34.347554    0.000000e+00
3           B_WORK  0.049100     3.460204e-03       14.189985    0.000000e+00
4        D_WORK_T1 -0.014494     4.185344e-03       -3.462972    5.342447e-04
5           B_MACM  0.101068     1.584645e-02        6.377949    1.794751e-10
6        D_MACM_T1 -0.073766     1.946042e-02       -3.790550    1.503138e-04
7            B_HHM -0.009063     4.272359e-03       -2.121333    3.389380e-02
8             B_AT -0.002781     1.210851e-03       -2.296401    2.165295e-02
9 

/tmp/ipykernel_6945/1419250103.py:20: DeprecationWarning: get_estimated_parameters is deprecated. Use get_pandas_estimated_parameters(estimation_results=my_results) instead
  pandas_results = results.get_estimated_parameters()
